# **Analyse de sentiments sur les critiques spectateurs sur Allociné**
**Projet Python pour la Data science - 2A ENSAE**

Zakaria BOULLIAIRE, Massyle DENDENE, Brian RAMESH

# Introduction

L'idée de ce projet est de prédire ("mettre la variable à prédire"), à partir d'une analyse de sentiment faite sur les critiques données pas les spectateurs (et non la presse), sur le site Allociné. 
Nous allons donc consituer une base de données de film, en scrappant le site Allociné et en utilisant l'API de The Movie Database (TMDB) pour compléter les données manquantes.

### Imports nécessaires

In [3]:
%%time
# BeutifulSoup pour le scrapping
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import requests

# Classique python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint

from warnings import warn
from time import sleep

import seaborn as sns

from datetime import datetime
from dateutil import parser 

CPU times: user 1.76 s, sys: 4.21 s, total: 5.98 s
Wall time: 1.02 s


# Collecte des données

### Scrapping sur Allociné

Tout d'abord, nous allons restreindre notre base de données au film américain des années 2010 à 2021. Cela constiuerait une base de données de 11 822 films.

Les données qui peuvent nous être utiles sont les suivantes : Titre original du film, identifiant du film sur Allociné (qui nous sera utile pour récupérer les critiques plus tard), la note des spectateurs et celle de la presse, le nombre de critiques presse, le nombre de critiques spectateurs et le nombre de votes pour la notes (dans une seule et même variable, qu'on spérera lors du nettoyage), la date de sortie du film, le budget et le Box Office US. 
Nous avions également commencer à scrapper le N°Visa du film, qui est unique pour chaque film, en vue d'utiliser cette donnée pour la collecte sur TMDB, mais il y avait énormément de film pour lesquels cette donnée manquait. Nous avons décider de ne pas l'utiliser.

L'idée de la fonction ci dessous est de scrapper les données dont on a besoin à partir de l'url et du nombre de page.

In [4]:
%%time
def scrapping_film_allocine(base_url, nb_page):

    # Liste ou on stock nos données
    data = []
    
    # Variable de comptage pour voir l'évolution du scrapping, et détecter les eventuelles erreurs
    i=0

    # Boucle sur les pages
    for page in range(1, nb_page+1): 
        url_page_ac = f"{base_url}{page}"

        response_page_ac = requests.get(url_page_ac)

        if response_page_ac.status_code == 200:
            bs_page_ac = bs(response_page_ac.text, "html.parser")
            films_page_ac = bs_page_ac.findAll("li", attrs={'class': "mdl"})

            for film_allocine in films_page_ac:
                i+=1
                try:
                    
                    # Id du film sur allo cine
                    meta_title_link = film_allocine.find('a', class_='meta-title-link')

                    if meta_title_link:
                        href1 = meta_title_link.get('href')
                        film_id = href1.split('=')[-1].split('.')[0]
                    else:
                        film_id = None


                    # Scrapping de la page fiche info du film qu'on obtient grace à l'id trouvé ci dessus
                    url_fiche_film = f'https://www.allocine.fr/film/fichefilm_gen_cfilm={film_id}.html'
                    response_fiche_film = requests.get(url_fiche_film)
                    bs_fiche_film = bs(response_fiche_film.text, "html.parser")

                    # Titre
                    titre_allocine = meta_title_link.text

                    span_titre_original = bs_fiche_film.find('span', class_='light', string='Titre original ')
                    titre_original = span_titre_original.find_next_sibling(string=True).strip() if span_titre_original else titre_allocine


                    # Notes spectateurs/presse, nombre critiques presse/spectateurs
                    bloc_notes = bs_fiche_film.findAll('span', class_='stareval-note')
                    list_notes = [notes.get_text(strip=True) for notes in bloc_notes]

                    if len(list_notes)==0:
                        note_presse = None
                        note_spectateur = None
                        
                    else:
                        index_delimiteur = list_notes.index('--')
                        new_liste_notes = list_notes[:index_delimiteur]

                        if len(new_liste_notes)==2:
                            note_presse = new_liste_notes[0]
                            note_spectateur = new_liste_notes[1]
                        
                        elif len(new_liste_notes) > 0 and len(new_liste_notes) <= 1:
                            note_spectateur = new_liste_notes[0]
                            note_presse = None

                    bloc_critiques = bs_fiche_film.find_all('span', class_='stareval-review')

                    if len(bloc_critiques)==2:
                        critiques_element_presse = bloc_critiques[0].text
                        critiques_element_spec = bloc_critiques[1].text
                    elif len(bloc_critiques) > 0 and len(bloc_critiques) <= 1:
                        critiques_element_spec = bloc_critiques[0].text
                        critiques_element_presse = None
                    else:
                        critiques_element_presse = None
                        critiques_element_spec = None


                    #Date, durée, budget
                    date_film_element = film_allocine.find('span', class_='date')
                    date_film = date_film_element.text if date_film_element else None

                    duree_film_element = bs_fiche_film.find('span', class_='spacer')
                    duree_film = duree_film_element.next_sibling.strip() if duree_film_element else None

                    budget_element = bs_fiche_film.find('span', class_='what light', string='Budget')
                    budget_film = budget_element.find_next('span').string if budget_element else None

                    #visa_element = bs_fiche_film.find('span', class_='what light', string='N° de Visa')
                    #visa_film = visa_element.find_next('span').string if visa_element else None

                    # Box office
                    url_box_office = f'https://www.allocine.fr/film/fichefilm-{film_id}/box-office/'
                    response_box_office = requests.get(url_box_office)
                    bs_box_office = bs(response_box_office.text, "html.parser")

                    cumul = bs_box_office.findAll('td', {'data-heading': 'Cumul'})
                    list_cumul = [cum.get_text(strip=True) for cum in cumul]

                    box_office_film = list_cumul[-1] if list_cumul else 'None'

                    data.append([titre_original, note_presse, note_spectateur, critiques_element_presse, critiques_element_spec, film_id, box_office_film, budget_film,
                                date_film, duree_film])
                    
                    df_data = pd.DataFrame(data, columns=["Titre original", "Note press", "Notes spectateur", "Critiques presse", "Critiques spectateurs", 'id allocine',
                                          'Box office', 'Budget', 'date', 'duree'])


                    print(i)
                except Exception as e:
                    print(f"Une erreur s'est produite pour le film {i} : {e}")

    return df_data

CPU times: user 4 µs, sys: 5 µs, total: 9 µs
Wall time: 13.8 µs


In [ ]:
%%time

# Urls qu'on veut scrapper
base_url_2010_2020, nb_page_2010_2020 = 'https://www.allocine.fr/films/pays-5002/decennie-2010/?page=', 646
base_url_2020, nb_page_2020 = 'https://www.allocine.fr/films/pays-5002/decennie-2020/annee-2020/?page', 66
base_url_2021, nb_page_2021 = 'https://www.allocine.fr/films/pays-5002/decennie-2020/annee-2021/?page' 77


# Application de la foncrion scrapping_film_allocine
df_data_2010_2020 = scrapping_film_allocine(base_url_2010_2020, nb_page_2010_2020)
df_data_2020 = scrapping_film_allocine(base_url_2020, nb_page_2020)
df_data_2021 = scrapping_film_allocine(base_url_2021, nb_page_2021)

# Créer un DataFrame avec les données collectées
dfs=[df_data_2010_2020, df_data_2020, df_data_2021]

df_film_ac = pd.concat(dfs, ignore_index=True)

In [ ]:
df_film_ac.to_csv('df_film_ac.csv', index=False)

### Nettoyage de la base obtenue

In [3]:
df_film_ac= pd.read_csv("df_film_ac.csv")

In [4]:
df_film_ac 

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree
0,Shutter Island,"3,8","4,4",31 critiques,80471 notes dont 4605 critiques,132039,127 770 000,80 000 000 $,24 février 2010,2h 17min
1,Inception,"4,1","4,5",24 critiques,110095 notes dont 7212 critiques,143692,290 948 208,160 000 000 $,21 juillet 2010,2h 28min
2,Harry Potter and the Deathly Hallows - Part 1,"3,2","4,0",20 critiques,52676 notes dont 2887 critiques,126693,291 377 000,150 000 000 $,24 novembre 2010,2h 26min
3,Prince of Persia: The Sands of Time,"2,6","3,1",22 critiques,26730 notes dont 2133 critiques,126678,89 981 000,200 000 000 $,26 mai 2010,2h 06min
4,The Book of Eli,"2,4","3,3",20 critiques,10503 notes dont 1144 critiques,128955,92 524 000,80 000 000 $,20 janvier 2010,1h 49min
...,...,...,...,...,...,...,...,...,...,...
11816,Amarillo,NaN,NaN,NaN,NaN,294872,NaN,-,NaN,NaN
11817,Chastise,NaN,NaN,NaN,NaN,295695,NaN,-,NaN,NaN
11818,Tapawingo,NaN,NaN,NaN,NaN,296176,NaN,-,NaN,NaN
11819,Mr. Birthday,NaN,NaN,NaN,NaN,298844,NaN,-,NaN,NaN


Comme on peut voir, la base a besoin d'être nettoyé. Nous allons extraires le nombre de critiques de la presse, le nombre de notes sepcteurs et le nombre de critiques des spectateurs. Nous allons convertir également le format de la date, et la durée du film en minute.

Puis, nous allons supprimer tous les films qui n'ont pas de note spectateurs et/ou qui ont moins de 5 critiques

In [5]:
print(df_film_ac.dtypes)

Titre original           object
Note press               object
Notes spectateur         object
Critiques presse         object
Critiques spectateurs    object
id allocine               int64
Box office               object
Budget                   object
date                     object
duree                    object
dtype: object


In [6]:
# Modifications des types 

df_film_ac['Box office'] = df_film_ac['Box office'].str.replace(' ', '').astype(float)
df_film_ac['Note press'] = df_film_ac['Note press'].str.replace(',', '.').astype(float)
df_film_ac['Notes spectateur'] = df_film_ac['Notes spectateur'].replace("--", np.nan)
df_film_ac['Notes spectateur'] = df_film_ac['Notes spectateur'].str.replace(',', '.').astype(float)

In [7]:
print(df_film_ac.dtypes)

Titre original            object
Note press               float64
Notes spectateur         float64
Critiques presse          object
Critiques spectateurs     object
id allocine                int64
Box office               float64
Budget                    object
date                      object
duree                     object
dtype: object


In [8]:
# Fonction pour convertir le mois en anglais
def french_to_english_month(month_french):
    months_mapping = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    return months_mapping.get(month_french.lower(), month_french)


# Fonction pour convertir la durée en minutes
def convert_duration(duration_str):
    if isinstance(duration_str, str):
        # Supprimer les espaces et diviser la chaîne en parties
        parts = duration_str.replace(' ', '').split('h')

        # Vérifier la présence des heures et des minutes
        if len(parts) == 2:
            hours = int(parts[0])
            minutes = 0 if 'min' not in parts[1] else int(parts[1].replace('min', ''))
            
            # Calculer la durée totale en minutes
            total_minutes = hours * 60 + minutes
            
            return int(total_minutes)
    
    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return float('nan')



# Fonction pouur extraire le nombre de critique de la presse 
def extract_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Utiliser isdigit() pour extraire uniquement les chiffres
        return np.nan if not critiques_str.split()[0].isdigit() else int(critiques_str.split()[0])
    else:
        return np.nan



# Fonction pour extraire le nombre de notes et le nombre de critiques
def extract_notes_and_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Trouver les nombres dans la chaîne
        numbers = [int(word) for word in critiques_str.split() if word.isdigit()]

        # Extraire le nombre de notes et de critiques en fonction de la longueur de la liste "numbers"
        if len(numbers) == 1:
            return numbers[0], np.nan
        elif len(numbers) == 2:
            return numbers[0], numbers[1]

    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return np.nan, np.nan

In [9]:
## Appliacation de nos fonctions


# Remplacer les chaînes "nan" par des valeurs NaN
df_film_ac['date'] = df_film_ac['date'].replace('nan', np.nan)

# Appliquer la fonction pour convertir le mois en anglais
df_film_ac['date'] = df_film_ac['date'].apply(lambda x: ' '.join([french_to_english_month(word) for word in str(x).split()]) if pd.notna(x) else np.nan)

# Utiliser dateutil.parser.parse pour convertir les dates en objets datetime
df_film_ac['date'] = df_film_ac['date'].apply(lambda x: parser.parse(x, dayfirst=True) if isinstance(x, str) else x)


# Appliquer la fonction de conversion pour la durée
df_film_ac['duree_minutes'] = df_film_ac['duree'].apply(convert_duration)

# Appliquer la fonction extract_notes_and_critiques_count
df_film_ac[['Nombre_de_notes_spectateurs', 'Nombre_de_critiques_spectateurs']] = df_film_ac['Critiques spectateurs'].apply(extract_notes_and_critiques_count).apply(pd.Series)

# Appliquer la fonction extract_critiques_count
df_film_ac['Nombre_de_critiques_presse'] = df_film_ac['Critiques presse'].apply(extract_critiques_count)

In [10]:
df_film_ac

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree,duree_minutes,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,Nombre_de_critiques_presse
0,Shutter Island,3.8,4.4,31 critiques,80471 notes dont 4605 critiques,132039,127770000.0,80 000 000 $,2010-02-24,2h 17min,137.0,80471.0,4605.0,31.0
1,Inception,4.1,4.5,24 critiques,110095 notes dont 7212 critiques,143692,290948208.0,160 000 000 $,2010-07-21,2h 28min,148.0,110095.0,7212.0,24.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,20 critiques,52676 notes dont 2887 critiques,126693,291377000.0,150 000 000 $,2010-11-24,2h 26min,146.0,52676.0,2887.0,20.0
3,Prince of Persia: The Sands of Time,2.6,3.1,22 critiques,26730 notes dont 2133 critiques,126678,89981000.0,200 000 000 $,2010-05-26,2h 06min,126.0,26730.0,2133.0,22.0
4,The Book of Eli,2.4,3.3,20 critiques,10503 notes dont 1144 critiques,128955,92524000.0,80 000 000 $,2010-01-20,1h 49min,109.0,10503.0,1144.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11816,Amarillo,NaN,NaN,NaN,NaN,294872,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11817,Chastise,NaN,NaN,NaN,NaN,295695,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11818,Tapawingo,NaN,NaN,NaN,NaN,296176,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11819,Mr. Birthday,NaN,NaN,NaN,NaN,298844,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN


In [11]:
# Retrait des colonnes inutiles 
colonnes_a_retirer = ["Critiques presse","Critiques spectateurs","duree","Budget"]
df_film_ac = df_film_ac.drop(colonnes_a_retirer,axis=1)

In [13]:
# Retrait de tous les films sans critique spectateurs
# On garde que les films avec au moins plus de 5 commentaires

df_film_ac_clean = df_film_ac.dropna(subset=['Nombre_de_critiques_spectateurs'])
df_film_ac_clean = df_film_ac_clean[df_film_avec_critiques['Nombre_de_critiques_spectateurs'] >= 5]

In [ ]:
# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_film_ac_clean['mois_sortie'] = df_film_ac_clean['date'].dt.month
df_film_ac_clean['annee_sortie'] = df_film_ac_clean['date'].dt.year

In [14]:
df_film_ac_clean

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,Nombre_de_critiques_presse
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,80471.0,4605.0,31.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,110095.0,7212.0,24.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,52676.0,2887.0,20.0
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,26730.0,2133.0,22.0
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,10503.0,1144.0,20.0
...,...,...,...,...,...,...,...,...,...,...
11101,The Gateway,NaN,2.3,273584,NaN,2022-03-16,91.0,40.0,6.0,NaN
11102,"Batman: The Long Halloween, Part Two",NaN,3.7,292641,NaN,2021-08-04,84.0,114.0,7.0,NaN
11104,Mortal Kombat Legends: Battle of the Realms,NaN,3.3,294101,NaN,2021-09-15,80.0,50.0,6.0,NaN
11117,Black As Night,NaN,1.6,287632,NaN,2021-10-01,87.0,53.0,6.0,NaN


In [ ]:
df_film_ac_clean.to_csv('df_film_ac_clean.csv', index=False)

### Scrapping des données sur TMDB

Le scrapping des données sur TMDB nécessite l'utilisation de l'API du site. Pour se faire, il a fallu créer une clé :

In [ ]:
key_api='d1d1413d8379729633d60e9f5cc4a730'

In [ ]:
## Fonctions

# Fonction pour récupérer l'id du film
def id_recup(titre):
    url_api=f"https://api.themoviedb.org/3/search/movie?api_key={key_api}&query={titre}" 
    req = requests.get(url_api)
    carte = req.json()

    ind=[]
    for film in range(len(carte['results'])):
        ind.append(carte['results'][film]['id'])
    return(ind)


# Fonction pour récuperer les infos du film à partir de l'id 
def df_avec_id(id):
    id_film= id
    url_new_api = f"https://api.themoviedb.org/3/movie/{id_film}?api_key={key_api}&language=en-US"
    req_new = requests.get(url_new_api)
    wb_new = req_new.json()
    
    
    #ajustement des données 
    if 'belongs_to_collection' in wb_new and wb_new['belongs_to_collection'] is not None:
        wb_new['belongs_to_collection'] = wb_new['belongs_to_collection']['name']

        
    # Ajustement des données pour la clé 'genres'
    if 'genres' in wb_new:
        wb_new['genres'] = ', '.join([x['name'] for x in wb_new['genres']])
    else:
        wb_new['genres'] = None

# Ajustement des données pour la clé 'production_companies'
    if 'production_companies' in wb_new:
        wb_new['production_companies'] = ', '.join([x['name'] for x in wb_new['production_companies']])
    else:
        wb_new['production_companies'] = None

# Ajustement des données pour la clé 'production_countries'
    if 'production_countries' in wb_new:
        wb_new['production_countries'] = ', '.join([x['name'] for x in wb_new['production_countries']])
    else:
        wb_new['production_countries'] = None

# Ajustement des données pour la clé 'spoken_languages'
    if 'spoken_languages' in wb_new:
        wb_new['spoken_languages'] = ', '.join([x['name'] for x in wb_new['spoken_languages']])
    else:
        wb_new['spoken_languages'] = None

    
    df=pd.DataFrame(wb_new, index=[0])
    
    return (df)


def get_movie_info(movie_id_list):
# Initialiser un DataFrame vide pour stocker les informations sur les films
    movie_df = pd.DataFrame()

    for movie_id in movie_id_list:
        # Utiliser la deuxième fonction pour obtenir les informations détaillées du film
        movie_info = df_avec_id(movie_id)

        # Vérifier si des informations ont été trouvées
        if movie_info is not None:
            # Ajouter les informations du film au DataFrame
            movie_df = pd.concat([movie_df, movie_info], ignore_index=True)

    return movie_df


def create_movie_list(movie_list):
    all_movies_df = pd.DataFrame()

    for movie_name in movie_list:
        movie_id_list = id_recup(movie_name)
        if movie_id_list:
            movie_info_df = get_movie_info(movie_id_list)
            # Ajouter les informations du film au DataFrame global
            all_movies_df = pd.concat([all_movies_df, movie_info_df], ignore_index=True)

    return all_movies_df



# Fonction qui prend en entrée une liste et qui renvoie 4 sous-liste de taille identique (à une division entière près) 
def diviser_liste(liste):
    taille = len(liste)
    quart = taille // 4
    partie1 = liste[:quart]
    partie2 = liste[quart:2*quart]
    partie3 = liste[2*quart:3*quart]
    partie4 = liste[3*quart:]
    return partie1, partie2, partie3, partie4

On récupère ensuite les films de la base d'Allociné nettoyé précédemment.

In [ ]:
df_film_ac_clean= pd.read_csv("df_film_ac_clean.csv")
liste_films_ac = df_film_ac_clean['Titre original'].tolist()

len(liste_films_ac)

On va séparer notre dataframe en 4 en vue de la récolte des données à l'aide de l'API de TMDB. En effet, on rencontrait une erreur liée aux nombres de requêtes lorsqu'on utilisait le data frame en entier (ou même si on le séparait en 2)

In [ ]:
liste_films_ac_1, liste_films_ac_2, liste_films_ac_3, liste_films_ac_4 = diviser_liste(liste_films_ac)


# Affichage des quatres listes résultantes
print("Partie 1:", len(liste_films_ac_1))
print("Partie 2:", len(liste_films_ac_2))
print("Partie 3:", len(liste_films_ac_3))
print("Partie 4:", len(liste_films_ac_4))

In [ ]:
%%time 
df_film_ac_clean_1 = create_movie_list(liste_films_ac_1)


# CPU times: user 3min 37s, sys: 17.1 s, total: 3min 54s
# Wall time: 21min 6s

In [ ]:
%%time
df_film_ac_clean_2 = create_movie_list(liste_films_ac_2)

# CPU times: user 3min 54s, sys: 20.2 s, total: 4min 14s
# Wall time: 21min 41s

In [ ]:
%%time
df_film_ac_clean_3 = create_movie_list(liste_films_ac_3)

# CPU times: user 3min 52s, sys: 19.9 s, total: 4min 12s
# Wall time: 20min 36s

In [ ]:
%%time
df_film_ac_clean_4 = create_movie_list(liste_films_ac_4)

# CPU times: user 3min 43s, sys: 19.5 s, total: 4min 04s
# Wall time: 20min 56s

In [ ]:
dataframes = [df_film_ac_clean_1, df_film_ac_clean_2, df_film_ac_clean_3, df_film_ac_clean_4]

# Rassembler les DataFrames en un seul DataFrame
df_tmdb = pd.concat(dataframes, ignore_index=True)

# Sauvegarder le DataFrame tmdb en CSV
df_tmdb.to_csv('df_tmdb.csv', index=False)

# Afficher le DataFrame final
df_tmdb.head()

### Merge de la base Allociné et de la base TMDB

In [ ]:
# Séparation du mois et de l'année de sortie dans les deux bases

# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_tmdb['mois_sortie'] = df_tmdb['release_date'].dt.month
df_tmdb['annee_sortie'] = df_tmdb['release_date'].dt.year

df_tmdb.head()

### Nettoyage de la base finale

Tout comme pour la base Allociné, nous allons procéder au nettoyage de la base obtenue après le merge des deux bases.

In [ ]:
df_tmdb= pd.read_csv("df_tmdb.csv", engine="python")
print(df_tmdb.dtypes)

In [ ]:
# On supprime les trois dernières colonnes du dataframe

colonnes_a_supprimer = ["success", "status_code", "status_message"]
df_tmdb_clean = df_tmdb.drop(columns=colonnes_a_supprimer)

In [ ]:
# Changement du format de la date

df_tmdb_clean['release_date'] = pd.to_datetime(df_tmdb['release_date'], errors="coerce")

### Scrapping des critiques sur Allocine

Maintenant que nous avons notre base finale, avec toutes les données, on peut faire le scrapping des critiques spectateurs sur Allociné grâce à l'id du film.

In [4]:
df_final_ac = pd.read_csv('df_merged.csv')
df_final_ac

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,mois_sortie_y,annee_sortie_y,nom_du_realisateur_y
0,10 000 days,NaN,1.5,239331,NaN,NaN,91.0,NaN,32.0,7.0,...,English,Released,NaN,"10,000 Days",False,2.800,15.0,11.0,2014.0,Eric Small
1,100 Degrees Below Zero,NaN,1.2,221267,NaN,2015-04-21,89.0,NaN,110.0,51.0,...,English,Released,Cold as Hell,100 Degrees Below Zero,False,4.400,112.0,3.0,2013.0,R.D. Braunstein
2,10 Cloverfield Lane,3.6,3.6,231144,67880730.0,2016-03-16,103.0,20.0,9399.0,735.0,...,English,Released,Monsters come in many forms.,10 Cloverfield Lane,False,6.984,7456.0,3.0,2016.0,Dan Trachtenberg
3,10 Minutes Gone,NaN,1.5,266802,NaN,2020-01-10,96.0,NaN,170.0,23.0,...,English,Released,Keep your enemies close,10 Minutes Gone,False,5.358,204.0,9.0,2019.0,Brian A. Miller
4,10x10,NaN,2.0,251534,NaN,2018-11-29,87.0,NaN,447.0,54.0,...,English,Released,There are some secrets we cannot escape,10x10,False,5.200,508.0,4.0,2018.0,Suzi Ewing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3562,Zombeavers,NaN,1.9,226998,NaN,2015-02-17,85.0,NaN,444.0,66.0,...,English,Released,You'll be dammed!,Zombeavers,False,4.903,650.0,10.0,2014.0,Jordan Rubin
3563,Zombie Apocalypse,NaN,1.3,197632,NaN,2015-04-07,87.0,NaN,153.0,46.0,...,"English, Português, Pусский",Released,Mankind Survived... But Not Alive.,Zombie Apocalypse,False,5.300,217.0,10.0,2011.0,Nick Lyon
3564,Zombieland: Double Tap,3.1,3.5,176293,66643981.0,2019-10-30,99.0,23.0,5501.0,329.0,...,English,Released,NaN,Zombieland: Double Tap,False,6.951,5274.0,10.0,2019.0,Ruben Fleischer
3565,Zookeeper,1.0,2.2,86789,75800690.0,2011-08-17,99.0,2.0,735.0,102.0,...,English,Released,Welcome to his jungle.,Zookeeper,False,5.429,1539.0,7.0,2011.0,Frank Coraci


In [5]:
df_final_ac['Nombre_de_critiques_spectateurs'] = df_final_ac['Nombre_de_critiques_spectateurs'].astype(int)
#df_final_ac = df_final_ac.sort_values(by='Nombre_de_critiques_spectateurs', ascending=False)

somme_critique = df_final_ac['Nombre_de_critiques_spectateurs'].sum()

liste_id = df_final_ac['id allocine'].tolist()
nb_critique = df_final_ac['Nombre_de_critiques_spectateurs'].tolist()
print(somme_critique)

800488


In [16]:
%%time

def scrapping_critique(liste_id, nb_critique):
    assert len(liste_id) == len(nb_critique)
    data = []
    nb_page=1
    # Boucle sur les pages
    k=0
    for j in range(len(liste_id)):
        i=0
        page=0

        while i!=nb_critique[j]:
            base_url = f"https://www.allocine.fr/film/fichefilm-{liste_id[j]}/critiques/spectateurs/?page=".format()
            page+=1
            url_page_critique = f"{base_url}{page}"
            response_page_critique = requests.get(url_page_critique)
            
            if response_page_critique.status_code == 200:
                bs_page_critique = bs(response_page_critique.text, "html.parser")
                films_page_critique = bs_page_critique.findAll("div", attrs={'class': "hred review-card cf"})

                for critique in films_page_critique:
                    try:

                        note_critique = critique.find('span', class_='stareval-note').text.strip()

                        date_publication = critique.find('span', class_='review-card-meta-date light').text.strip()
                        critique = critique.find('div', class_='content-txt review-card-content').text.strip()

                        data.append([liste_id[j], note_critique, date_publication, critique])
                        i+=1
                        k+=1
                        print(f' {k} / {sum(nb_critique)} '.format())
                        df_critique = pd.DataFrame(data, columns=["id_allocine", "Note de la critique", "Date de publication", 'Critique'])
                    except Exception as e:
                        print(f"Une erreur s'est produite pour le commentaire {j} : {e}")
    
    return df_critique

CPU times: user 6 µs, sys: 3 µs, total: 9 µs
Wall time: 15.3 µs


In [7]:
## on split nos deux listes en 4

def diviser_liste(liste):
    taille = len(liste)
    quart = taille // 4
    partie1 = liste[:quart]
    partie2 = liste[quart:2*quart]
    partie3 = liste[2*quart:3*quart]
    partie4 = liste[3*quart:]
    return partie1, partie2, partie3, partie4

In [17]:
%%time

liste_id_1, liste_id_2, liste_id_3, liste_id_4 = diviser_liste(liste_id)
nb_critique_1, nb_critique_2, nb_critique_3, nb_critique_4 = diviser_liste(nb_critique)
#print(sum(nb_critique_4))

CPU times: user 116 µs, sys: 63 µs, total: 179 µs
Wall time: 186 µs


In [18]:
%%time

scrapping_critique_1 = scrapping_critique(liste_id_1, nb_critique_1)

 1 / 157431 
 2 / 157431 
 3 / 157431 
 4 / 157431 
 5 / 157431 
 6 / 157431 
 7 / 157431 
 8 / 157431 
 9 / 157431 
 10 / 157431 
 11 / 157431 
 12 / 157431 
 13 / 157431 
 14 / 157431 
 15 / 157431 
 16 / 157431 
 17 / 157431 
 18 / 157431 
 19 / 157431 
 20 / 157431 
 21 / 157431 
 22 / 157431 
 23 / 157431 
 24 / 157431 
 25 / 157431 
 26 / 157431 
 27 / 157431 
 28 / 157431 
 29 / 157431 
 30 / 157431 
 31 / 157431 
 32 / 157431 
 33 / 157431 
 34 / 157431 
 35 / 157431 
 36 / 157431 
 37 / 157431 
 38 / 157431 
 39 / 157431 
 40 / 157431 
 41 / 157431 
 42 / 157431 
 43 / 157431 
 44 / 157431 
 45 / 157431 
 46 / 157431 
 47 / 157431 
 48 / 157431 
 49 / 157431 
 50 / 157431 
 51 / 157431 
 52 / 157431 
 53 / 157431 
 54 / 157431 
 55 / 157431 
 56 / 157431 
 57 / 157431 
 58 / 157431 
 59 / 157431 
 60 / 157431 
 61 / 157431 
 62 / 157431 
 63 / 157431 
 64 / 157431 
 65 / 157431 
 66 / 157431 
 67 / 157431 
 68 / 157431 
 69 / 157431 
 70 / 157431 
 71 / 157431 
 72 / 157431 
 

KeyboardInterrupt: 

In [ ]:
df_scrapping_critique_1.to_csv('scrapping_critique_1.csv', index=False)


In [36]:
scrapping_critique_1

,id_allocine,Note de la critique,Date de publication,Critique
0,239331,"0,5",Publiée le 5 octobre 2015,"""10.000 days"" est à la base une série créée pa..."
1,239331,"1,0",Publiée le 12 décembre 2015,Les effets spéciaux bas de gamme mettent les a...
2,239331,"1,0",Publiée le 10 janvier 2017,"10,000 somnifères. A peine ça commence qu'un g..."
3,239331,"0,5",Publiée le 24 mai 2016,"""10 000 days"" est d'une nullité abyssale. Tout..."
4,239331,"1,5",Publiée le 1 janvier 2017,"Beaucoup de parlotes, pas d'action ni d'effets..."
...,...,...,...,...
788,231144,"3,0",Publiée le 20 mars 2016,Bof !!!Déçue de la tournure du film et des inc...
789,231144,"4,5",Publiée le 22 février 2017,un très bon film avec rebondissements qui touc...
790,231144,"4,5",Publiée le 11 août 2016,Je pense que beaucoup n'ont pas compris que l'...
791,231144,"4,0",Publiée le 30 avril 2016,"10 Cloverfield Lane est un film... Etrange, tr..."


In [ ]:
%%time

scrapping_critique_2 = scrapping_critique(liste_id_2, nb_critique_2)

In [ ]:
df_scrapping_critique_2.to_csv('scrapping_critique_2.csv', index=False)


In [ ]:
%%time

scrapping_critique_3 = scrapping_critique(liste_id_3, nb_critique_3)

In [ ]:
df_scrapping_critique_3.to_csv('scrapping_critique_3.csv', index=False)


In [ ]:
%%time

scrapping_critique_4 = scrapping_critique(liste_id_4, nb_critique_4)

In [ ]:
df_scrapping_critique_4.to_csv('scrapping_critique_4.csv', index=False)


In [ ]:
## On merge les 4 dataframes en un
dfs = [scrapping_critique_1, scrapping_critique_2, scrapping_critique_3, scrapping_critique_4]

# Rassembler les DataFrames en un seul DataFrame
df_critiques = pd.concat(dfs, ignore_index=True)

# Sauvegarder le DataFrame des critiques en CSV
df_critiques.to_csv('df_critiques.csv', index=False)

# Afficher le DataFrame final
#df_critiques.head()